# Menú de Índice

1. [Abre y examina el archivo de datos](#introduccion)
2. [Abre y examina el archivo de datos](#abre-y-examina-el-archivo-de-datos)
3. [Segmentación de datos](#segmenta)
4. [Investiga la calidad de diferentes modelos cambiando los hiperparámetros](#calidad)
5. [Evaluar el modelo usando el conjunto de prueba](#conjunto-de-prueba)
6. [Prueba de cordura](#cordura)
6. [Conclusión](#conclusion)

## 1. Introducción <a id='introduccion'></a>

En un mercado tan dinámico como el de las telecomunicaciones, la capacidad de adaptación y anticipación es fundamental para mantenerse relevante. En este sentido, la compañía móvil Megaline se enfrenta a un desafío significativo: la persistencia de clientes que aún utilizan planes heredados. Con el objetivo de optimizar su cartera de servicios y satisfacer las necesidades cambiantes de sus usuarios, Megaline busca desarrollar un modelo predictivo capaz de analizar el comportamiento de sus clientes y recomendar adecuadamente uno de sus nuevos planes: Smart o Ultra.

A través de datos exhaustivos sobre el comportamiento de los suscriptores que ya han migrado a los planes nuevos, obtenidos previamente en el proyecto del sprint de Análisis Estadístico de Datos, se plantea la tarea de clasificación para crear un modelo efectivo que pueda seleccionar el plan más apropiado para cada cliente. Con el análisis previo de datos ya realizado, se abre la oportunidad de centrarse en la creación y evaluación de modelos, brindando a Megaline la posibilidad de tomar decisiones informadas y estratégicas para mejorar la experiencia del cliente y mantener su competitividad en el mercado móvil en constante evolución.

## 2. Importación de librerias e importación de datos<a id='abre-y-examina-el-archivo-de-datos'></a>


In [1]:
#Librerias
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
#->datos
# Leer el archivo y almacenarlo en df
def cargar_archivos_con_separadores(archivos_con_separadores):
    """
    Carga múltiples archivos CSV con separadores específicos para cada archivo y los almacena en un diccionario de DataFrames.
    
    :param archivos_con_separadores: Diccionario donde las claves son los nombres de los archivos y los valores son los separadores.
    :return: Diccionario con nombres de archivos como claves y DataFrames como valores.
    """
    df = {}  # Objeto que almacenará los DataFrames

    # Iterar sobre los archivos y sus separadores
    for archivo_nombre, separador in archivos_con_separadores.items():
        url_no_windows = '/datasets/' + archivo_nombre
        url_windows = 'datasets/' + archivo_nombre
        
        try:
            # Intentar cargar desde la ruta no-Windows
            if os.path.exists(url_no_windows):
                df[archivo_nombre] = pd.read_csv(url_no_windows, sep=separador, dtype={'votes': 'Int64'})
                print(f"Archivo cargado exitosamente desde '{url_no_windows}'")
            
            # Intentar cargar desde la ruta Windows
            elif os.path.exists(url_windows):
                df[archivo_nombre] = pd.read_csv(url_windows, sep=separador, dtype={'votes': 'Int64'})
                print(f"Archivo cargado exitosamente desde '{url_windows}'")
            
            else:
                raise FileNotFoundError(f"Error: el archivo '{archivo_nombre}' no se encuentra en ninguna de las rutas especificadas.")
        
        except FileNotFoundError as e:
            print(e)
        
        except pd.errors.ParserError as e:
            print(f"Error al procesar el archivo CSV '{archivo_nombre}': {e}")
        
        except Exception as e:
            print(f"Ha ocurrido un error inesperado con el archivo '{archivo_nombre}': {e}")

    return df

# Ejemplo de uso
archivos_con_separadores = {
    'users_behavior.csv': ',',    
}

# Cargar los archivos en el objeto df
df_all = cargar_archivos_con_separadores(archivos_con_separadores)

# El objeto df ahora contiene un DataFrame por cada archivo cargado
df = df_all.get('users_behavior.csv')

 


Archivo cargado exitosamente desde 'datasets/users_behavior.csv'


In [3]:
# Examinar los primeros registros del dataset
print(df.head())
print(df.info())
print(df.describe())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246  

## 3. Segmentación de datos <a id='segmenta'></a>

In [4]:
# < separa los datos en entrenamiento y validación >
#df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345)

# Separar las características y la etiqueta
X = df.drop('is_ultra', axis=1)  # Reemplaza 'plan' con el nombre de la columna de etiquetas si es diferente
y = df['is_ultra']  # Reemplaza 'plan' con el nombre de la columna de etiquetas si es diferente

# Dividir el conjunto de datos en entrenamiento, validación y prueba
#60% para el data set de entrenamiento
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=12345)
# y del 40$ restante, 20% para validacion y 20% para prueba
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345)


print(f"Dimensiones del conjunto de entrenamiento: {X_train.shape}")
print(f"Dimensiones del conjunto de validación: {X_valid.shape}")
print(f"Dimensiones del conjunto de prueba: {X_test.shape}")

Dimensiones del conjunto de entrenamiento: (1285, 4)
Dimensiones del conjunto de validación: (964, 4)
Dimensiones del conjunto de prueba: (965, 4)


## 4. Investigación de la calidad de diferentes modelos <a id='calidad'></a>

In [5]:
mejor_precision = 0
mejor_modelo = None
mejor_profundidad = 0
resultados = []

# Probar diferentes profundidades de árbol de decisión
for profundidad in range(1, 21):
    modelo = DecisionTreeClassifier(max_depth=profundidad, random_state=12345)
    modelo.fit(X_train, y_train)
    predicciones_valid = modelo.predict(X_valid)
    precision_valid = accuracy_score(y_valid, predicciones_valid)
    resultados.append((profundidad, precision_valid))
    
    if precision_valid > mejor_precision:
        mejor_precision = precision_valid
        mejor_modelo = modelo
        mejor_profundidad = profundidad

# Mostrar los resultados
for profundidad, precision in resultados:
    print(f"Profundidad: {profundidad}, Precisión en validación: {precision:.4f}")

print(f"\nMejor profundidad: {mejor_profundidad}")
print(f"Mejor precisión en validación: {mejor_precision:.4f}")

Profundidad: 1, Precisión en validación: 0.7770
Profundidad: 2, Precisión en validación: 0.7822
Profundidad: 3, Precisión en validación: 0.8102
Profundidad: 4, Precisión en validación: 0.8039
Profundidad: 5, Precisión en validación: 0.7946
Profundidad: 6, Precisión en validación: 0.8050
Profundidad: 7, Precisión en validación: 0.7956
Profundidad: 8, Precisión en validación: 0.8008
Profundidad: 9, Precisión en validación: 0.7905
Profundidad: 10, Precisión en validación: 0.7770
Profundidad: 11, Precisión en validación: 0.7842
Profundidad: 12, Precisión en validación: 0.7749
Profundidad: 13, Precisión en validación: 0.7770
Profundidad: 14, Precisión en validación: 0.7666
Profundidad: 15, Precisión en validación: 0.7656
Profundidad: 16, Precisión en validación: 0.7500
Profundidad: 17, Precisión en validación: 0.7479
Profundidad: 18, Precisión en validación: 0.7407
Profundidad: 19, Precisión en validación: 0.7355
Profundidad: 20, Precisión en validación: 0.7417

Mejor profundidad: 3
Mejor p

## 5. Evaluar el modelo usando el conjunto de prueba <a id='conjunto-de-prueba'></a>

In [6]:
# Predecir en el conjunto de prueba
predicciones_test = mejor_modelo.predict(X_test)
precision_test = accuracy_score(y_test, predicciones_test)

print(f"Precisión en el conjunto de prueba: {precision_test:.4f}")

# Verificar si la precisión cumple con el umbral
umbral_exactitud = 0.75
if precision_test >= umbral_exactitud:
    print("El modelo cumple con el umbral de exactitud.")
else:
    print("El modelo no cumple con el umbral de exactitud.")

Precisión en el conjunto de prueba: 0.7772
El modelo cumple con el umbral de exactitud.


## 6. Prueba de cordura <a id='cordura'></a>

In [7]:
# se realiza una prueba de cordura comparando el modelo con un modelo trivial
# para asegurarnos de que el modelo no está simplemente memorizando los datos de entrenamiento.
# Modelo trivial (siempre predice la clase más frecuente)
modelo_dummy = DummyClassifier(strategy="most_frequent", random_state=12345)
modelo_dummy.fit(X_train, y_train)
predicciones = modelo_dummy.predict(X_test)
precision_dummy = accuracy_score(y_test, predicciones)

print(f"Precisión del modelo trivial en el conjunto de prueba: {precision_dummy:.4f}")

# Comprobar que el modelo real tiene un mejor desempeño que el modelo trivial
assert precision_test > precision_dummy, "El modelo no está mejor que el modelo trivial"
print("El modelo pasó la prueba de cordura y es mejor que un modelo trivial.")

Precisión del modelo trivial en el conjunto de prueba: 0.6984
El modelo pasó la prueba de cordura y es mejor que un modelo trivial.


## 7. Conclusión <a id='conclusion'></a>

Conclusión
Segmentación de datos: Dividimos los datos en conjuntos de entrenamiento, validación y prueba.
Investigación de modelos: Probamos diferentes profundidades del árbol de decisiones y encontramos la mejor profundidad basada en la precisión del conjunto de validación.
Evaluación del modelo: Evaluamos el modelo en el conjunto de prueba y verificamos que cumple con el umbral de precisión del 0.77.
Prueba de cordura: Confirmamos que el modelo tiene un mejor desempeño que un modelo trivial.